# Load Data <a class="anchor" id="acquire"></a>

Steps are:
    - Obtain data
        - scrape website
        - sql query
        - use given data
    - Load data
    

## 1. Obtain data

#### Scrape Website

In [ ]:
# Import packages
import requests
from bs4 import BeautifulSoup

# Specify url: url
url = 'https://www.python.org/~guido/'

# Package the request, send the request and catch the response: r
r = requests.get(url)

# Extract the response as html: html_doc
html_doc = r.text

# OR
# read from website
df = pd.read_html('http://www.fdic.gov/bank/individual/failed/banklist.html')

# Create a BeautifulSoup object from the HTML: soup
soup = BeautifulSoup(html_doc)

# Get the title of Guido's webpage: guido_title
guido_title = soup.title

# Print the title of Guido's webpage to the shell
print(guido_title)

# Get Guido's text: guido_text
guido_text = soup.text

# Print Guido's text to the shell
print(guido_text)

# Find all 'a' tags (which define hyperlinks): a_tags
a_tags = soup.find_all('a')

# Print the URLs to the shell
for link in a_tags:
    print(link.get('href'))

#### SQL query

# 2. Load Dataset

##### ENSURE IN CORRECT WORKING DIRECTORY

In [ ]:
# get working directory
os.getcwd()

# change working directory 
os.chdir('C:\\Users\\sexto\\Downloads')

In [ ]:
# if manually downloading data, can check files in directory to ensure it is there
import os
wd = os.getcwd()
os.listdir(wd)

##### DOWNLOAD

In [ ]:
# Make code as reproducible as possible by loading data from the direct link from the web
trainUrl = "https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftrain.csv"
df2 = pd.read_csv(trainUrl, sep=',')

# However, reading in this file caused problems. I am unsure why? It was supposed to be clean data
# Got an answer from stackoverflow from below which is tried dropping rows with errors but decided not to do this
train = pd.read_csv("https://www.kaggle.com/c/titanic/download/GQf0y8ebHO0C4JXscPPp%2Fversions%2FXkNkvXwqPPVG0Qt3MtQT%2Ffiles%2Ftrain.csv", error_bad_lines=False)

###### Size of data is important
- Look at the SIZE of the dataset(s) (usually train). Look at when not compressed.
- 4 GB is quite large. A GB is 1,000,000 KB.
- In kaggle, click on grid icon and can see the size of the dataset if click on the file.
- Otherwise, after downloading, click properties over the file name and can see the size.
- If file is very large then must read in only part of it. There is a way to break this up like the earthquake file.
- train = train.sample(1000000)

Different packages have their own way of reading data. 
- Pandas
- Dask
- Datatable
- Rapids

Apart from methods of reading data from the raw csv files, it is also common to convert the dataset into another format which uses lesser disk space, is smaller in size and/or can be read faster for subsequent reads. The file types explored in the notebook (Default csv and rest alphabetically):
- csv
- feather
- hdf5
- jay
- parquet
- pickle

Note that just reading data is not the end of the story. The final decision of which method to use should also consider the downstream tasks and processes of the data that will be required to run. 

##### PANDAS
Pandas is probably the most popular method of reading datasets and is also the default on Kaggle. It has a lot of options, flexibility and functions for reading and processing data.

One of the challenges with using pandas for reading large datasets is it's conservative nature while infering data types of the columns of a dataset often resulting in unnecessary large memory usage for the pandas dataframe. You can pre-define optimal data types of the columns (based on prior knowledge or sample inspection) and provide it explicitly while reading the dataset.

Documentation: https://pandas.pydata.org/docs/

In [ ]:
# Setting header to None ensures headers are not imported in IF you dont want them

import pandas as pd

%%time

# set names of columns AND data type in the column
dtypes = {
    "row_id": "int64",
    "user_id": "int32",
    "task_container_id": "int16",
    "user_answer": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

data = pd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes)

# look at header to ensure loaded correctly
data.head()

##### IMPORT EXCEL SHEETS WITH PANDAS

In [ ]:
# IMPORT EXCEL SHEETS WITH PANDAS
import pandas as pd

# Assign spreadsheet filename: file
file = 'battledeath.xlsx'

# Load spreadsheet: xls
xls = pd.ExcelFile(file)

# Print sheet names
print(xls.sheet_names)

# Load a sheet into a DataFrame by name of sheet
df1 = xls.parse('2004')

# Load a sheet into a DataFrame by index
df2 = xls.parse(0)

# Parse the first column of the second sheet and rename the column
df2 = xls.parse(1, usecols=[0], skiprows=[0], names=['Country'])

In [ ]:
df = pd.read_html('http://www.fdic.gov/bank/individual/failed/banklist.html')

##### DASK
Dask provides a framework to scale pandas workflows natively using a parallel processing architecture. For those of you who have used Spark, you will find an uncanny similarity between the two.
​
Documentation: https://docs.dask.org/en/latest/

In [ ]:
import dask.dataframe as dd

dtypes = {
    "row_id": "int64",
    "user_id": "int32",
    "task_container_id": "int16",
    "user_answer": "int8",
    "prior_question_elapsed_time": "float32", 
    "prior_question_had_explanation": "boolean"
}

data = dd.read_csv("../input/riiid-test-answer-prediction/train.csv", dtype=dtypes).compute()

print("Train size:", data.shape)

##### DATATABLE
Datatable (heavily inspired by R's data.table) can read large datasets fairly quickly and is often faster than pandas. It is specifically meant for data processing of tabular datasets with emphasis on speed and support for large sized data.

Documentation: https://datatable.readthedocs.io/en/latest/index.html

In [ ]:
# datatable installation with internet
# !pip install datatable==0.11.0 > /dev/null

# datatable installation without internet
!pip install ../input/python-datatable/datatable-0.11.0-cp37-cp37m-manylinux2010_x86_64.whl > /dev/null

import datatable as dt

%%time

data = dt.fread("../input/riiid-test-answer-prediction/train.csv")

print("Train size:", data.shape)

# read and change to pandas dataframe at same time
# data = dt.fread("../input/riiid-test-answer-prediction/train.csv").to_pandas()

##### RAPIDS
Rapids is a great option to scale data processing on GPUs. With a lot of machine learning modelling moving to GPUs, Rapids enables to build end-to-end data science solutions on one or more GPUs.

Documentation: https://docs.rapids.ai/

In [ ]:
# rapids installation (make sure to turn on GPU)
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path

import cudf

%%time

data = cudf.read_csv("../input/riiid-test-answer-prediction/train.csv")

print("Train size:", data.shape)

##### RELATIONAL DATABASE
A relational database is a type of database that stores and provides access to data points that are related to one another. ... The columns of the table hold attributes of the data, and each record usually has a value for each attribute, making it easy to establish the relationships among data points.

In [ ]:
# Import necessary module
from sqlalchemy import create_engine

# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Save the table names to a list: table_names
table_names = engine.table_names()

# Print the table names to the shell
print(table_names)

# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM Album", engine)

# Perform query and save results to DataFrame: df
# Save dataframe column names to corresponding names of table columns
with engine.connect() as con:
    rs = con.execute("SELECT Title, Name FROM Album INNER JOIN Artist on Album.ArtistID = Artist.ArtistID")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

##### CONVERT DATA TO ANOTHER FORMAT

In [ ]:
# reading data from csv using datatable and converting to pandas
data = dt.fread("../input/riiid-test-answer-prediction/train.csv").to_pandas()

# writing dataset as csv
data.to_csv("riiid_train.csv", index=False)

# writing dataset as hdf5
# HDF5 is a high-performance data management suite to store, manage and process large and complex data.
data.to_hdf("riiid_train.h5", "riiid_train")

# writing dataset as feather
# It is common to store data in feather (binary) format specifically for pandas. 
# It significantly improves reading speed of datasets.
data.to_feather("riiid_train.feather")

# writing dataset as parquet
# In the Hadoop ecosystem, parquet was popularly used as the primary file format for tabular datasets 
# and is now extensively used with Spark.
data.to_parquet("riiid_train.parquet")

# writing dataset as pickle
# Python objects can be stored in the form of pickle files and pandas has inbuilt functions
# to read and write dataframes as pickle objects.
data.to_pickle("riiid_train.pkl.gzip")

# writing dataset as jay
# Datatable uses .jay (binary) format which makes reading datasets blazing fast. 
dt.Frame(data).to_jay("riiid_train.jay")

##### Getting stock data from internet

In [ ]:
start = datetime.datetime(2006, 1, 1)
end = datetime.datetime(2016, 1, 1)

# Bank of America
BAC = data.DataReader("BAC", 'google', start, end)

# Could also do this for a Panel Object
df = data.DataReader(['BAC', 'C', 'GS', 'JPM', 'MS', 'WFC'],'google', start, end)